In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
pip install qdrant-client openai Pillow tqdm anthropic

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

import openai
from PIL import Image
import numpy as np
import json
import os

import os
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, PointStruct
from qdrant_client.http.models import VectorParams, Distance

# from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm
import io




In [10]:
from tensorflow.keras.models import load_model

# Load the pretrained CNN model
cnn_model = load_model('/content/drive/MyDrive/Dental Model/Dental_CNN_model_v4.h5')


In [11]:
cnn_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 254, 254, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 125, 125, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 60, 60, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 28, 28, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      25,690,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │              

 Total params: 26,116,037 (99.62 MB)

 Trainable params: 26,114,051 (99.62 MB)

 Non-trainable params: 1,984 (7.75 KB)

 Optimizer params: 2 (12.00 B)

In [12]:
cnn_model.pop()
cnn_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 254, 254, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 125, 125, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 60, 60, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 28, 28, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      25,690,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │              

 Total params: 26,115,842 (99.62 MB)

 Trainable params: 26,113,856 (99.62 MB)

 Non-trainable params: 1,984 (7.75 KB)

 Optimizer params: 2 (12.00 B)

In [ ]:
# Connect to Qdrant instance
qdrant_client = QdrantClient(
    url="quadrant-host-url",
    api_key="quadrant-api-key",
)

# # Delete the existing collection
# qdrant_client.delete_collection(collection_name="dental_images")

# # Define collection parameters
# collection_name = "dental_images"
# dimension = 64    # 64-dimensional image embeddings

# # Create collection in Qdrant
# qdrant_client.recreate_collection(
#     collection_name=collection_name,
#     vectors_config= VectorParams(size=dimension, distance=Distance.COSINE)

# )

In [14]:
# dataset_path = "/content/drive/MyDrive/dataset-RAG"
# for root, dirs, files in os.walk(dataset_path):
#     print(f"root: {root}")
#     print(f"dirs: {dirs}")
#     print(f"files: {files}")
#     print("-" * 20)
#     for file in files:
#         if file.endswith(".png") or file.endswith(".jpg"):
#             image_path = os.path.join(root, file)
#             print(image_path)
#             break

In [15]:
from PIL import Image
import numpy as np

# Load and preprocess the image
def preprocess_image(image_path):
    img = Image.open(image_path)

    # Convert image to RGB if it has an alpha channel
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Resize image and normalize pixel values
    img = img.resize((256, 256))
    img = np.array(img)

    return np.expand_dims(img, axis=0)  # Add batch dimension

In [22]:
collection_name = "dental_images"
# Function to retrieve the most similar image-description pair from Qdrant
def retrieve_similar_image_description(query_image_path, cnn_model):
    query_image_data = preprocess_image(query_image_path)
    query_embedding = cnn_model.predict(query_image_data)[0]  # Shape: (128,)

    # Perform a search in Qdrant for the most similar image embedding
    search_result = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=1  # Retrieve the most similar image
    )

    # Get the image-description pair
    retrieved_point = search_result[0]
    similar_image_path = retrieved_point.payload['image_path']
    description = retrieved_point.payload['response']

    return similar_image_path, description




In [ ]:
import anthropic

client = anthropic.Anthropic(api_key="claude-api-key")

def generate_answer(question, description):
    prompt = f"""
    Image description: {description}
    Question: {question}

    Please provide a detailed answer to the question based on the given image description and classification information.
    """

    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=1000,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return message.content


In [24]:
# Full workflow for receiving an image and question from the user
def handle_user_request(query_image_path, user_question):
    # Step 1: Retrieve the most similar image-description pair
    retrieved_image_path, retrieved_description = retrieve_similar_image_description(query_image_path, cnn_model)

    # Step 2: Generate an answer to the user's question based on the retrieved description
    answer = generate_answer(user_question, retrieved_description)

    return retrieved_image_path, retrieved_description, answer




In [26]:
from google.colab import files
query_image = files.upload()
query_image_name = next(iter(query_image))
import os
query_image_path = os.path.join('/content', query_image_name)
print("Full file path:", query_image_path)

Saving occlusion_vestibulaire_10.png to occlusion_vestibulaire_10.png
Full file path: /content/occlusion_vestibulaire_10.png


In [27]:

user_question = "What is the main dental issue in this image and how many aligners are needed?" # Ask your Question

retrieved_image, description, answer = handle_user_request(query_image_path, user_question)
print(f"Retrieved Image: {retrieved_image}")
print(f"Description: {description}")
print(f"Answer: {answer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Retrieved Image: /content/drive/My Drive/dataset-RAG/blanc/11-15/vue_occlusale_inf_30.png
Description: Classe : Blanc, Nombre d'aligneurs nécessaires : 11-15, Raison : Ce patient présente un encombrement modéré des dents antérieures sans désalignements extrêmes. La courbe de Spee est plate et les rotations dentaires sont inférieures à 15°. Ces facteurs indiquent une correction relativement simple qui ne nécessite que 11 à 15 aligneurs.
Answer: [TextBlock(text='Based on the image description and classification information provided, the main dental issue is moderate crowding of the anterior teeth without extreme misalignments. The specifics mentioned are:\n\n1. The patient presents with moderate crowding of the anterior (front) teeth.\n2. There are no extreme misalignments or dental rotations exceeding 15 degrees.\n3. The curve of Spee (the curve formed by the biting surfaces of the teeth) is flat.\n\nThese factors indicate a relatively straightforwa